In [1]:
import tweepy, json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO

# Authintication
consumer_key = 'dtl5siVf87HewDiYC6xMCWJQW'
consumer_secret = 'QrkmqeENVQYJwpMkSD7Abga5LyUBWAHl38P0ORCoD9hSf3vGqL'

access_token = '3305208076-j3a4iQw4NqpvUmdydZTMazZXbR0PEGej8QfGrex'
access_token_secret = 'vuYVpQE5Lo5oZd5r85BGcdPZpLK7RZdL44uG8DaHj3KMl'

print('Connecting to Twitter API....')
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Connecting to Twitter API....


In [6]:
# get the tweets
search_results = api.search(q="faceapp", lang='en', count=100)

In [42]:
def download(search_results):
    for i, tweet in enumerate(search_results):
        for j,media in enumerate(tweet.entities.get("media",[{}])):
            #checks if there is any media-entity
            if media.get("type",None) == "photo":
                # checks if the entity is of the type "photo"
                image_content=requests.get(media["media_url"])
                # save images
                img = Image.open(BytesIO(image_content.content))
                img.save(f"data/{str(tweet.id)}_{str(j)}.png")

In [45]:
searchQuery = '#faceapp'  # this is what we're searching for
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
tweetCount = 0
max_id = -1
sinceId = None

while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, lang='en', count=tweetsPerQry)
            else:
                new_tweets = api.search(q=searchQuery, lang='en', count=tweetsPerQry,
                                        since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, lang='en', count=tweetsPerQry,
                                        max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, lang='en', count=tweetsPerQry,
                                        max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
        #for tweet in new_tweets:
        download(new_tweets)
            
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets
Downloaded 1100 tweets


KeyboardInterrupt: 